# OpenAI SDK を使った Cohere AI API の呼び出し

https://docs.cohere.com/docs/compatibility-api

In [ ]:
import os
from dotenv import load_dotenv
import cohere
from openai import OpenAI

In [ ]:
load_dotenv()

client = OpenAI(
    base_url="https://api.cohere.ai/compatibility/v1",
    api_key=os.environ['COHERE_API_KEY'],
)

# Basic chat completion - チャットの基本

In [ ]:
completion = client.chat.completions.create(
    model="command-a-03-2025",
    messages=[
        {
            "role": "user",
            "content": "プログラミングの再帰処理について、俳句を作って。",
        },
    ],
)

print(completion.choices[0].message)


# Chat with Streaming - チャットのストリーミング処理

In [ ]:
stream = client.chat.completions.create(
    model="command-a-03-2025",
    messages=[
        {
            "role": "user",
            "content": "オブジェクト指向プログラミングのクラス設計について、短歌を作って。",
        },
    ],
    stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")


# State management - 状態の管理

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "developer",
            "content": "マスター忍者のように応答すること。",
        },
        {
            "role": "user",
            "content": "2 + 2　は？",
        },
        {
            "role": "assistant",
            "content": "Arrr, matey! 2 + 2 be 4, just like a doubloon in the sea!",
        },
        {
            "role": "user",
            "content": "それに 30 足すと？",
        },
    ],
    model="command-a-03-2025",
)

print(completion.choices[0].message)


# Structured Output - 構造化した出力

In [ ]:
completion = client.beta.chat.completions.parse(
    model="command-a-03-2025",
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON describing a book.",
        }
    ],
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "author": {"type": "string"},
                "publication_year": {"type": "integer"},
            },
            "required": ["title", "author", "publication_year"],
        },
    },
)

print(completion.choices[0].message.content)


# ツールの利用 - Tool use

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flight_info",
            "description": "Get flight information between two cities or airports",
            "parameters": {
                "type": "object",
                "properties": {
                    "loc_origin": {
                        "type": "string",
                        "description": "The departure airport, e.g. MIA",
                    },
                    "loc_destination": {
                        "type": "string",
                        "description": "The destination airport, e.g. NYC",
                    },
                },
                "required": ["loc_origin", "loc_destination"],
            },
        },
    }
]

messages = [
    {"role": "developer", "content": "Today is April 30th"},
    {
        "role": "user",
        "content": "When is the next flight from Miami to Seattle?",
    },
    {
        "role": "assistant",
        "tool_calls": [
            {
                "function": {
                    "arguments": '{ "loc_destination": "Seattle", "loc_origin": "Miami" }',
                    "name": "get_flight_info",
                },
                "id": "get_flight_info0",
                "type": "function",
            }
        ],
    },
    {
        "role": "tool",
        "name": "get_flight_info",
        "tool_call_id": "get_flight_info0",
        "content": "Miami to Seattle, May 1st, 10 AM.",
    },
]

completion = client.chat.completions.create(
    model="command-a-03-2025",
    messages=messages,
    tools=tools,
    temperature=0.7,
)

print(completion.choices[0].message)


# Embedding - 埋め込みパラメータ

In [ ]:
response = client.embeddings.create(
    input=["Hello world!"],
    model="embed-v4.0",
    encoding_format="float",
)

print(
    response.data[0].embedding[:5]
)  # Display the first 5 dimensions
